# Evaluation of accumulation results

In [2]:
import os, sys

import pcl

import numpy as np

from tqdm import tqdm

import re

sys.path.append(os.path.abspath('../../Dataset/scripts') )
from utils import *

sys.path.append( os.path.abspath('../model') )
from train import *
import model as MODEL

In [6]:
""" Parameters """
GPU_INDEX = 0
NUM_POINT = 256
NUM_CLASSES = 3

BATCH_SIZE = 1

# model_path = os.path.abspath('../notebook/train_log_20181020/model/backup/model.ckpt')
model_path = os.path.abspath('../notebook/train_log_20181020/model/model.ckpt')


In [17]:
""" Load data """

data_path = os.path.abspath('../../Dataset/evaluation_acc_cls')

pcd_file_list = []
for root, dirs, files in os.walk(data_path):
    rootpath = os.path.join(os.path.abspath(data_path), root)

    for file in files:
        if file.split('.')[-1] == 'pcd':

            filepath = os.path.join(rootpath, file)
            pcd_file_list.append(filepath)

af_frame = 6

data = [[] for _ in range(af_frame + 1)]
acc_frame_list = []

## Regular expression
re_af = re.compile("AF_\d{4}")

for pcd_file in tqdm(pcd_file_list):
    cloud = np.asarray(pcl.load(pcd_file))
    
    mat_results = re_af.search(pcd_file)
    af =  int( mat_results.group(0).split('_')[-1] )
    
    if af <= 6:
        data[af].append(cloud)
        
sys.stdout.flush()
print len(data)
for i in range( len(data) ): print i, len(data[i])

100%|██████████| 386/386 [00:00<00:00, 582.03it/s]

7
0 0
1 234
2 98
3 24
4 16
5 10
6 4


In [34]:
""" Data visualization """
%matplotlib qt

acc_frame_sel = 5
idx = 0

start_idx = (6 * 6) * idx
end_idx = start_idx + 6*6

clouds = data[acc_frame_sel]

plt.close('all')
fig = plt.figure()

fig.subplots_adjust(left=0.02,top= 0.98,bottom=0.02,right=0.98,wspace=0.1,hspace=0.5)

ax=fig.add_subplot(111, projection='3d')
ax = display_point_cloud_box_ax(ax, clouds[idx]) 

point_num = clouds[idx].shape[0]

ax.set_title('idx: {} point_num: {}'.format(idx, point_num))

figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()
plt.show()

In [53]:
is_training = False

# reset graph
tf.reset_default_graph()

with tf.device('/gpu:'+str(GPU_INDEX)):
    pointclouds_pl, labels_pl = MODEL.placeholder_inputs(BATCH_SIZE, NUM_POINT)
    is_training_pl = tf.placeholder(tf.bool, shape=())

    # simple model
    pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl)
    loss = MODEL.get_loss(pred, labels_pl, end_points)

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()

# Create a session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = True

ops = {'pointclouds_pl': pointclouds_pl,
       'labels_pl': labels_pl,
       'is_training_pl': is_training_pl,
       'pred': pred,
       'loss': loss}

# ops.reset_default_graph() 

sess = tf.Session(config=config)

# Restore variables from disk.
saver.restore(sess, model_path)

print "Model restored"
sys.stdout.flush()

results_list = []
for i in range(len(data)):
    
    print "Dataset {} / {} ".format(i+1, len(data))
    sys.stdout.flush()
    
    cd_list = []
    for d in data[i]:
        cd = NormalizeResample(d, NUM_POINT)
        cd_list.append(cd)
        
    current_data = np.asarray(cd_list)
        
    current_label = np.array([0 for _ in range( current_data.shape[0] )])
    
    num_batches = current_data.shape[0] // BATCH_SIZE
    
    print "Number of total dataset / evaluated dataset: {} / {}".format(current_data.shape[0], num_batches * BATCH_SIZE)
    sys.stdout.flush()
    
    results = []

    for batch_idx in trange(num_batches):
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE

        feed_dict = {ops['pointclouds_pl']: current_data[start_idx:end_idx].reshape((BATCH_SIZE, NUM_POINT, 3)),
                     ops['labels_pl']: current_label[start_idx:end_idx].reshape(BATCH_SIZE),
                     ops['is_training_pl']: is_training}
        loss_val, pred_val = sess.run([ops['loss'], ops['pred']], feed_dict=feed_dict )

        pred_val = np.argmax(pred_val, 1)
        results.extend(pred_val)
    
    results_list.append(results)


INFO:tensorflow:Summary name classify loss is illegal; using classify_loss instead.
INFO:tensorflow:Summary name mat loss is illegal; using mat_loss instead.
INFO:tensorflow:Restoring parameters from /media/kyungpyo/SmartCarContest/MasterThesisProject/Network/notebook/train_log_20181020/model/model.ckpt
Model restored
Dataset 1 / 7 
Number of total dataset / evaluated dataset: 0 / 0


0it [00:00, ?it/s]

Dataset 2 / 7 
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256


100%|██████████| 234/234 [00:06<00:00, 34.71it/s]

Dataset 3 / 7 
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
Number of total dataset / evaluated dataset: 98 / 98


...


100%|██████████| 16/16 [00:00<00:00, 25.72it/s]

Dataset 6 / 7 
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
(256, 3)
Number of total dataset / evaluated dataset: 10 / 10



100%|██████████| 10/10 [00:00<00:00, 26.97it/s]

Dataset 7 / 7 
(256, 3)
(256, 3)
(256, 3)
(256, 3)
Number of total dataset / evaluated dataset: 4 / 4



100%|██████████| 4/4 [00:00<00:00, 24.25it/s]


In [1]:
for i, results in enumerate(results_list):
    print len(results)
    np_re = np.array(results)
    if np_re.shape[0] == 0: continue
    print i, float ( np.sum(np_re == np.ones(np_re.shape[0])) ) / float ( np_re.shape[0] ) * 100.
    

NameError: name 'results_list' is not defined